In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
item_categories = pd.read_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_item_categories.csv')
items = pd.read_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_items.csv')
sales_train = pd.read_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_sales_train.csv')
sample = pd.read_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/sample_submission.csv')
shops = pd.read_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_shops.csv')

In [3]:
df = pd.read_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_df.csv')
test = pd.read_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/test.csv')
data = pd.read_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_all_data.csv')
data_encoded = pd.read_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_all_data_encoded.csv')

# grouped by date... sum of item_cnt_month

In [5]:
a = list(data.groupby('date_block_num').sum()['item_cnt_month'].index)
b = list(data.groupby('date_block_num').sum()['item_cnt_month'])
d = pd.DataFrame(list(zip(a,b)), columns=['date_block_num','date_sum_cnt'])
d['date_block_num'] += 1
q = dict(zip(list(d['date_block_num']),list(d['date_sum_cnt'])))
data['date_sum_cnt'] = data['date_block_num'].map(q)
data.loc[data['date_block_num']==0,'date_sum_cnt'] = data[data['date_block_num']==12]['date_sum_cnt'].mean()

In [4]:
def lag_feature(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

In [6]:
group = data.groupby(['date_block_num']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_avg_item_cnt' ]
group.reset_index(inplace=True)

In [9]:
data = pd.merge(data, group, on=['date_block_num'], how='left')
data['date_avg_item_cnt'] = data['date_avg_item_cnt'].astype(np.float16)
data = lag_feature(data, [1], 'date_avg_item_cnt')
data.drop(['date_avg_item_cnt'], axis=1, inplace=True)

In [10]:
group = data.groupby(['date_block_num', 'shop_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_shop_avg_item_cnt' ]
group.reset_index(inplace=True)

data = pd.merge(data, group, on=['date_block_num','shop_id'], how='left')
data['date_shop_avg_item_cnt'] = data['date_shop_avg_item_cnt'].astype(np.float16)
data = lag_feature(data, [1,2,3,6,12], 'date_shop_avg_item_cnt')
data.drop(['date_shop_avg_item_cnt'], axis=1, inplace=True)

# date_sum_shop_cnt

In [11]:
i = data.groupby(['date_block_num','shop_id']).sum()['item_cnt_month'].index
df = i.to_frame(index = False)
df['date_sum_shop_cnt'] = list(data.groupby(['date_block_num','shop_id']).sum()['item_cnt_month'])
df['date_block_num'] += 1

In [12]:
data = pd.merge(data, df, on=['date_block_num','shop_id'], how='left')

In [13]:
data.loc[data['date_block_num']==0,'date_sum_shop_cnt'] = data[data['date_block_num']==12]['date_sum_shop_cnt'].median()

In [14]:
data_encoded = data.copy()
data_encoded['city'], _ = pd.factorize(data_encoded['city'])
data_encoded['type'], _ = pd.factorize(data_encoded['type'])
data_encoded['sub_type'], _ = pd.factorize(data_encoded['sub_type'])
data_encoded.to_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_all_data_encoded.csv', index=False)

In [15]:
data_encoded

,date_block_num,shop_id,item_id,item_cnt_month,month,year,city,item_category_id,type,sub_type,...,date_sum_cnt,date_avg_item_cnt_x,date_avg_item_cnt_y,date_avg_item_cnt_lag_1,date_shop_avg_item_cnt_lag_1,date_shop_avg_item_cnt_lag_2,date_shop_avg_item_cnt_lag_3,date_shop_avg_item_cnt_lag_6,date_shop_avg_item_cnt_lag_12,date_sum_shop_cnt
0,0.0,2,27,1.0,1,2013,0,19,0,0,...,160313.0,2.022035,2.022035,NaN,NaN,NaN,NaN,NaN,NaN,3395.0
1,0.0,2,33,1.0,1,2013,0,37,1,1,...,160313.0,2.022035,2.022035,NaN,NaN,NaN,NaN,NaN,NaN,3395.0
2,0.0,2,317,1.0,1,2013,0,45,2,2,...,160313.0,2.022035,2.022035,NaN,NaN,NaN,NaN,NaN,NaN,3395.0
3,0.0,2,438,1.0,1,2013,0,45,2,2,...,160313.0,2.022035,2.022035,NaN,NaN,NaN,NaN,NaN,NaN,3395.0
4,0.0,2,471,2.0,1,2013,0,49,2,3,...,160313.0,2.022035,2.022035,NaN,NaN,NaN,NaN,NaN,NaN,3395.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821799,34.0,45,18454,0.0,11,2015,17,55,6,10,...,61583.0,0.000000,0.000000,1.953125,1.47168,NaN,NaN,NaN,NaN,686.0
1821800,34.0,45,16188,0.0,11,2015,17,64,5,27,...,61583.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,686.0
1821801,34.0,45,15757,0.0,11,2015,17,55,6,10,...,61583.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,686.0
1821802,34.0,45,19648,0.0,11,2015,17,40,1,8,...,61583.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,686.0


In [16]:
i = data_encoded.groupby(['date_block_num','item_id']).mean()['item_cnt_month'].index
df = i.to_frame(index = False)
df['date_block_num_item_meanencoded'] = list(data_encoded.groupby(['date_block_num','item_id']).mean()['item_cnt_month'])

In [17]:
df['date_block_num'] += 1

In [18]:
df['test'] = df['date_block_num'].astype('str') + '_' + df['item_id'].astype('str')

In [19]:
d = dict(zip(list(df['test']), list(df['date_block_num_item_meanencoded'])))

In [20]:
data['test'] = data['date_block_num'].astype('str') + '_' + data['item_id'].astype('str')

In [21]:
data['date_item_meanencoded'] = data['test'].map(d)

In [22]:
data.drop('test',axis=1,inplace=True)

In [23]:
data = data.fillna(0)

In [24]:
data_encoded = data.copy()
data_encoded['city'], _ = pd.factorize(data_encoded['city'])
data_encoded['type'], _ = pd.factorize(data_encoded['type'])
data_encoded.to_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_all_data_encoded.csv', index=False)

In [25]:
data_encoded['sub_type'], _ = pd.factorize(data_encoded['sub_type'])

In [26]:
data_encoded

,date_block_num,shop_id,item_id,item_cnt_month,month,year,city,item_category_id,type,sub_type,...,date_avg_item_cnt_x,date_avg_item_cnt_y,date_avg_item_cnt_lag_1,date_shop_avg_item_cnt_lag_1,date_shop_avg_item_cnt_lag_2,date_shop_avg_item_cnt_lag_3,date_shop_avg_item_cnt_lag_6,date_shop_avg_item_cnt_lag_12,date_sum_shop_cnt,date_item_meanencoded
0,0.0,2,27,1.0,1,2013,0,19,0,0,...,2.022035,2.022035,0.000000,0.00000,0.0,0.0,0.0,0.0,3395.0,0.0
1,0.0,2,33,1.0,1,2013,0,37,1,1,...,2.022035,2.022035,0.000000,0.00000,0.0,0.0,0.0,0.0,3395.0,0.0
2,0.0,2,317,1.0,1,2013,0,45,2,2,...,2.022035,2.022035,0.000000,0.00000,0.0,0.0,0.0,0.0,3395.0,0.0
3,0.0,2,438,1.0,1,2013,0,45,2,2,...,2.022035,2.022035,0.000000,0.00000,0.0,0.0,0.0,0.0,3395.0,0.0
4,0.0,2,471,2.0,1,2013,0,49,2,3,...,2.022035,2.022035,0.000000,0.00000,0.0,0.0,0.0,0.0,3395.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821799,34.0,45,18454,0.0,11,2015,17,55,6,10,...,0.000000,0.000000,1.953125,1.47168,0.0,0.0,0.0,0.0,686.0,1.0
1821800,34.0,45,16188,0.0,11,2015,17,64,5,27,...,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,686.0,1.0
1821801,34.0,45,15757,0.0,11,2015,17,55,6,10,...,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,686.0,1.0
1821802,34.0,45,19648,0.0,11,2015,17,40,1,8,...,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,686.0,1.0


In [27]:
data_encoded.groupby(['date_block_num','item_id'])['item_cnt_month'].apply(lambda x: x.cumsum())

0          1.0
1          1.0
2          1.0
3          1.0
4          2.0
          ... 
1821799    0.0
1821800    0.0
1821801    0.0
1821802    0.0
1821803    0.0
Name: item_cnt_month, Length: 1821804, dtype: float64

In [28]:
data_encoded['test'] = data_encoded.groupby(['date_block_num','item_id'])['item_cnt_month'].cumsum()

In [29]:
data_encoded

,date_block_num,shop_id,item_id,item_cnt_month,month,year,city,item_category_id,type,sub_type,...,date_avg_item_cnt_y,date_avg_item_cnt_lag_1,date_shop_avg_item_cnt_lag_1,date_shop_avg_item_cnt_lag_2,date_shop_avg_item_cnt_lag_3,date_shop_avg_item_cnt_lag_6,date_shop_avg_item_cnt_lag_12,date_sum_shop_cnt,date_item_meanencoded,test
0,0.0,2,27,1.0,1,2013,0,19,0,0,...,2.022035,0.000000,0.00000,0.0,0.0,0.0,0.0,3395.0,0.0,1.0
1,0.0,2,33,1.0,1,2013,0,37,1,1,...,2.022035,0.000000,0.00000,0.0,0.0,0.0,0.0,3395.0,0.0,1.0
2,0.0,2,317,1.0,1,2013,0,45,2,2,...,2.022035,0.000000,0.00000,0.0,0.0,0.0,0.0,3395.0,0.0,1.0
3,0.0,2,438,1.0,1,2013,0,45,2,2,...,2.022035,0.000000,0.00000,0.0,0.0,0.0,0.0,3395.0,0.0,1.0
4,0.0,2,471,2.0,1,2013,0,49,2,3,...,2.022035,0.000000,0.00000,0.0,0.0,0.0,0.0,3395.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821799,34.0,45,18454,0.0,11,2015,17,55,6,10,...,0.000000,1.953125,1.47168,0.0,0.0,0.0,0.0,686.0,1.0,0.0
1821800,34.0,45,16188,0.0,11,2015,17,64,5,27,...,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,686.0,1.0,0.0
1821801,34.0,45,15757,0.0,11,2015,17,55,6,10,...,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,686.0,1.0,0.0
1821802,34.0,45,19648,0.0,11,2015,17,40,1,8,...,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,686.0,1.0,0.0


In [30]:
data_encoded[data_encoded['date_block_num']==33]['test'].describe()

count    31531.000000
mean        20.277917
std         44.183158
min          0.000000
25%          3.000000
50%          7.000000
75%         20.000000
max        768.000000
Name: test, dtype: float64

In [31]:
data_encoded[data_encoded['item_cnt_month']<0]

,date_block_num,shop_id,item_id,item_cnt_month,month,year,city,item_category_id,type,sub_type,...,date_avg_item_cnt_y,date_avg_item_cnt_lag_1,date_shop_avg_item_cnt_lag_1,date_shop_avg_item_cnt_lag_2,date_shop_avg_item_cnt_lag_3,date_shop_avg_item_cnt_lag_6,date_shop_avg_item_cnt_lag_12,date_sum_shop_cnt,date_item_meanencoded,test


In [32]:
data_encoded.iloc[1601084]

date_block_num                      33.000000
shop_id                             48.000000
item_id                           6674.000000
item_cnt_month                       0.000000
month                               10.000000
year                              2015.000000
city                                29.000000
item_category_id                    12.000000
type                                 9.000000
sub_type                            26.000000
ID                                   0.000000
item_cnt_month_lag_1                 0.000000
item_cnt_month_lag_2                 2.000000
item_cnt_month_lag_3                 0.000000
item_cnt_month_lag_6                 0.000000
item_cnt_month_lag_12                0.000000
date_sum_cnt                     57675.000000
date_avg_item_cnt_x                  1.953094
date_avg_item_cnt_y                  1.953094
date_avg_item_cnt_lag_1              0.000000
date_shop_avg_item_cnt_lag_1         0.000000
date_shop_avg_item_cnt_lag_2      

In [39]:
group = data_encoded.groupby(['date_block_num']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_avg_item_cnt' ]
group.reset_index(inplace=True)

In [40]:
data_encoded = pd.merge(data_encoded, group, on=['date_block_num'], how='left')
data_encoded['date_avg_item_cnt'] = data_encoded['date_avg_item_cnt'].astype(np.float16)
data_encoded = lag_feature(data_encoded, [1], 'date_avg_item_cnt')

In [41]:
group['date_block_num'] += 1

In [42]:
d = dict(zip(list(group['date_block_num']),list(group['date_avg_item_cnt'])))
data_encoded['date_avg_item_cnt_lag_1'] = data['date_block_num'].map(d)

In [43]:
m = data_encoded[data_encoded['date_block_num']==12]['date_avg_item_cnt_lag_1'].mean()
data_encoded.loc[data_encoded['date_block_num']==0,'date_avg_item_cnt_lag_1'] = m

In [44]:
data_encoded

,date_block_num,shop_id,item_id,item_cnt_month,month,year,city,item_category_id,type,sub_type,...,date_shop_avg_item_cnt_lag_2,date_shop_avg_item_cnt_lag_3,date_shop_avg_item_cnt_lag_6,date_shop_avg_item_cnt_lag_12,date_sum_shop_cnt,date_item_meanencoded,test,date_avg_item_cnt_lag_1_y,date_avg_item_cnt,date_avg_item_cnt_lag_1
0,0.0,2,27,1.0,1,2013,0,19,0,0,...,0.0,0.0,0.0,0.0,3395.0,0.0,1.0,NaN,2.021484,2.418906
1,0.0,2,33,1.0,1,2013,0,37,1,1,...,0.0,0.0,0.0,0.0,3395.0,0.0,1.0,NaN,2.021484,2.418906
2,0.0,2,317,1.0,1,2013,0,45,2,2,...,0.0,0.0,0.0,0.0,3395.0,0.0,1.0,NaN,2.021484,2.418906
3,0.0,2,438,1.0,1,2013,0,45,2,2,...,0.0,0.0,0.0,0.0,3395.0,0.0,1.0,NaN,2.021484,2.418906
4,0.0,2,471,2.0,1,2013,0,49,2,3,...,0.0,0.0,0.0,0.0,3395.0,0.0,2.0,NaN,2.021484,2.418906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821799,34.0,45,18454,0.0,11,2015,17,55,6,10,...,0.0,0.0,0.0,0.0,686.0,1.0,0.0,1.953125,0.000000,1.953094
1821800,34.0,45,16188,0.0,11,2015,17,64,5,27,...,0.0,0.0,0.0,0.0,686.0,1.0,0.0,NaN,0.000000,1.953094
1821801,34.0,45,15757,0.0,11,2015,17,55,6,10,...,0.0,0.0,0.0,0.0,686.0,1.0,0.0,NaN,0.000000,1.953094
1821802,34.0,45,19648,0.0,11,2015,17,40,1,8,...,0.0,0.0,0.0,0.0,686.0,1.0,0.0,NaN,0.000000,1.953094


In [45]:
group = data_encoded.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_avg_item_cnt' ]
group.reset_index(inplace=True)

In [46]:
for i in [1,2,3,6,12]:
    temp = group.copy()
    temp['date_block_num'] += i
    temp['date_item_avg_item_cnt' + '_lag_' + str(i)] = temp['date_item_avg_item_cnt']
    temp.drop('date_item_avg_item_cnt',axis=1,inplace=True)
    data_encoded = pd.merge(data_encoded, temp, on=['date_block_num','item_id'], how='left')

In [47]:
group = data_encoded.groupby(['date_block_num', 'shop_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_shop_avg_item_cnt' ]
group.reset_index(inplace=True)

In [48]:
for i in [1,2,3,6,12]:
    temp = group.copy()
    temp['date_block_num'] += i
    temp['date_shop_avg_item_cnt' + '_lag_' + str(i)] = temp['date_shop_avg_item_cnt']
    temp.drop('date_shop_avg_item_cnt',axis=1,inplace=True)
    data_encoded = pd.merge(data_encoded, temp, on=['date_block_num','shop_id'], how='left')

In [49]:
data_encoded

,date_block_num,shop_id,item_id,item_cnt_month,month,year,city,item_category_id,type,sub_type,...,date_item_avg_item_cnt_lag_1,date_item_avg_item_cnt_lag_2,date_item_avg_item_cnt_lag_3,date_item_avg_item_cnt_lag_6,date_item_avg_item_cnt_lag_12,date_shop_avg_item_cnt_lag_1_y,date_shop_avg_item_cnt_lag_2_y,date_shop_avg_item_cnt_lag_3_y,date_shop_avg_item_cnt_lag_6_y,date_shop_avg_item_cnt_lag_12_y
0,0.0,2,27,1.0,1,2013,0,19,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,2,33,1.0,1,2013,0,37,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,2,317,1.0,1,2013,0,45,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,2,438,1.0,1,2013,0,45,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,2,471,2.0,1,2013,0,49,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821799,34.0,45,18454,0.0,11,2015,17,55,6,10,...,1.0,1.00,1.000000,1.300000,NaN,1.472103,1.597561,1.405941,1.502066,1.851373
1821800,34.0,45,16188,0.0,11,2015,17,64,5,27,...,1.0,1.00,NaN,NaN,NaN,1.472103,1.597561,1.405941,1.502066,1.851373
1821801,34.0,45,15757,0.0,11,2015,17,55,6,10,...,1.0,1.00,1.000000,1.222222,1.125,1.472103,1.597561,1.405941,1.502066,1.851373
1821802,34.0,45,19648,0.0,11,2015,17,40,1,8,...,1.0,1.00,1.166667,1.000000,NaN,1.472103,1.597561,1.405941,1.502066,1.851373


In [50]:
group = data_encoded.groupby(['date_block_num', 'item_category_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_cat_avg_item_cnt' ]
group.reset_index(inplace=True)

In [51]:
for i in [1,2]:
    temp = group.copy()
    temp['date_block_num'] += i
    temp['date_cat_avg_item_cnt' + '_lag_' + str(i)] = temp['date_cat_avg_item_cnt']
    temp.drop('date_cat_avg_item_cnt',axis=1,inplace=True)
    data_encoded = pd.merge(data_encoded, temp, on=['date_block_num','item_category_id'], how='left')

In [52]:
group = data_encoded.groupby(['date_block_num', 'shop_id', 'item_category_id']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_cat_avg_item_cnt']
group.reset_index(inplace=True)

data_encoded = pd.merge(data_encoded, group, on=['date_block_num', 'shop_id', 'item_category_id'], how='left')
data_encoded['date_shop_cat_avg_item_cnt'] = data_encoded['date_shop_cat_avg_item_cnt'].astype(np.float16)
data_encoded = lag_feature(data_encoded, [1], 'date_shop_cat_avg_item_cnt')

In [53]:
group = data_encoded.groupby(['date_block_num', 'shop_id', 'type']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_type_avg_item_cnt']
group.reset_index(inplace=True)

In [54]:
for i in [1,2]:
    temp = group.copy()
    temp['date_block_num'] += i
    temp['date_shop_type_avg_item_cnt' + '_lag_' + str(i)] = temp['date_shop_type_avg_item_cnt']
    temp.drop('date_shop_type_avg_item_cnt',axis=1,inplace=True)
    data_encoded = pd.merge(data_encoded, temp, on=['date_block_num','shop_id','type'], how='left')

In [58]:
group = data_encoded.groupby(['date_block_num', 'shop_id', 'type']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_type_avg_item_cnt']
group.reset_index(inplace=True)

data_encoded = pd.merge(data_encoded, group, on=['date_block_num', 'shop_id', 'type'], how='left')
data_encoded['date_shop_type_avg_item_cnt'] = data_encoded['date_shop_type_avg_item_cnt'].astype(np.float16)
data_encoded = lag_feature(data_encoded, [1], 'date_shop_type_avg_item_cnt')
data_encoded.drop(['date_shop_type_avg_item_cnt'], axis=1, inplace=True)

In [59]:
group = data_encoded.groupby(['date_block_num', 'shop_id', 'sub_type']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_subtype_avg_item_cnt']
group.reset_index(inplace=True)

data_encoded = pd.merge(data_encoded, group, on=['date_block_num', 'shop_id', 'sub_type'], how='left')
data_encoded['date_shop_subtype_avg_item_cnt'] = data_encoded['date_shop_subtype_avg_item_cnt'].astype(np.float16)
data_encoded = lag_feature(data_encoded, [1], 'date_shop_subtype_avg_item_cnt')
data_encoded.drop(['date_shop_subtype_avg_item_cnt'], axis=1, inplace=True)

In [60]:
group = data_encoded.groupby(['date_block_num', 'city']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_city_avg_item_cnt' ]
group.reset_index(inplace=True)

In [61]:
for i in [1,2,3,6,12]:
    temp = group.copy()
    temp['date_block_num'] += i
    temp['date_city_avg_item_cnt' + '_lag_' + str(i)] = temp['date_city_avg_item_cnt']
    temp.drop('date_city_avg_item_cnt',axis=1,inplace=True)
    data_encoded = pd.merge(data_encoded, temp, on=['date_block_num','city'], how='left')

In [62]:
group = data_encoded.groupby(['date_block_num', 'item_id', 'city']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_city_avg_item_cnt' ]
group.reset_index(inplace=True)

In [63]:
for i in [1,2,3,6,12]:
    temp = group.copy()
    temp['date_block_num'] += i
    temp['date_item_city_avg_item_cnt' + '_lag_' + str(i)] = temp['date_item_city_avg_item_cnt']
    temp.drop('date_item_city_avg_item_cnt',axis=1,inplace=True)
    data_encoded = pd.merge(data_encoded, temp, on=['date_block_num', 'item_id', 'city'], how='left')

In [64]:
group = data_encoded.groupby(['date_block_num', 'type']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_type_avg_item_cnt' ]
group.reset_index(inplace=True)

In [65]:
for i in [1,2,3,6,12]:
    temp = group.copy()
    temp['date_block_num'] += i
    temp['date_type_avg_item_cnt' + '_lag_' + str(i)] = temp['date_type_avg_item_cnt']
    temp.drop('date_type_avg_item_cnt',axis=1,inplace=True)
    data_encoded = pd.merge(data_encoded, temp, on=['date_block_num', 'type'], how='left')

In [66]:
group = data_encoded.groupby(['date_block_num', 'sub_type']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_subtype_avg_item_cnt' ]
group.reset_index(inplace=True)

In [67]:
for i in [1,2,3,6,12]:
    temp = group.copy()
    temp['date_block_num'] += i
    temp['date_subtype_avg_item_cnt' + '_lag_' + str(i)] = temp['date_subtype_avg_item_cnt']
    temp.drop('date_subtype_avg_item_cnt',axis=1,inplace=True)
    data_encoded = pd.merge(data_encoded, temp, on=['date_block_num', 'sub_type'], how='left')

In [68]:
data_encoded.to_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_all_data_encoded.csv', index=False)

In [69]:
group = sales_train.groupby(['item_id']).agg({'item_price': ['mean']})
group.columns = ['item_avg_item_price']
group.reset_index(inplace=True)

In [70]:
data_encoded = pd.merge(data_encoded, group, on=['item_id'], how='left')
data_encoded['item_avg_item_price'] = data_encoded['item_avg_item_price'].astype(np.float16)

In [71]:
group = sales_train.groupby(['date_block_num','item_id']).agg({'item_price': ['mean']})
group.columns = ['date_item_avg_item_price']
group.reset_index(inplace=True)

In [72]:
data_encoded = pd.merge(data_encoded, group, on=['date_block_num','item_id'], how='left')
data_encoded['date_item_avg_item_price'] = data_encoded['date_item_avg_item_price'].astype(np.float16)

In [73]:
def lag_feature(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

In [74]:
lags = [1,2,3,4,5,6]
data_encoded = lag_feature(data_encoded, lags, 'date_item_avg_item_price')

In [76]:
for i in lags:
    data_encoded['delta_price_lag_'+str(i)] = (data_encoded['date_item_avg_item_price_lag_'+str(i)] - data_encoded['item_avg_item_price']) / data_encoded['item_avg_item_price']

In [77]:
def select_trend(row):
    for i in lags:
        if row['delta_price_lag_'+str(i)]:
            return row['delta_price_lag_'+str(i)]
    return 0
    
data_encoded['delta_price_lag'] = data_encoded.apply(select_trend, axis=1)
data_encoded['delta_price_lag'] = data_encoded['delta_price_lag'].astype(np.float16)
data_encoded['delta_price_lag'].fillna(0, inplace=True)

In [78]:
fetures_to_drop = ['item_avg_item_price', 'date_item_avg_item_price']
for i in lags:
    fetures_to_drop += ['date_item_avg_item_price_lag_'+str(i)]
    fetures_to_drop += ['delta_price_lag_'+str(i)]

data_encoded.drop(fetures_to_drop, axis=1, inplace=True)

In [79]:
sales_train['revenue'] = sales_train['item_price'] * sales_train['item_cnt_day']

In [80]:
sales_train.to_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_sales_train.csv', index=False)

In [81]:
group = sales_train.groupby(['date_block_num','shop_id']).agg({'revenue': ['sum']})
group.columns = ['date_shop_revenue']
group.reset_index(inplace=True)

In [82]:
data_encoded = pd.merge(data_encoded, group, on=['date_block_num','shop_id'], how='left')
data_encoded['date_shop_revenue'] = data_encoded['date_shop_revenue'].astype(np.float32)

In [83]:
group = group.groupby(['shop_id']).agg({'date_shop_revenue': ['mean']})
group.columns = ['shop_avg_revenue']
group.reset_index(inplace=True)

In [84]:
data_encoded = pd.merge(data_encoded, group, on=['shop_id'], how='left')
data_encoded['shop_avg_revenue'] = data_encoded['shop_avg_revenue'].astype(np.float32)

In [85]:
data_encoded['delta_revenue'] = (data_encoded['date_shop_revenue'] - data_encoded['shop_avg_revenue']) / data_encoded['shop_avg_revenue']
data_encoded['delta_revenue'] = data_encoded['delta_revenue'].astype(np.float16)

In [86]:
data_encoded = lag_feature(data_encoded, [1], 'delta_revenue')

In [87]:
data_encoded.drop(['date_shop_revenue','shop_avg_revenue','delta_revenue'], axis=1, inplace=True)

In [88]:
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])

In [89]:
data_encoded['month'] -=1 

In [90]:
data_encoded['days'] = data_encoded['month'].map(days).astype(np.int8)

In [91]:
data_encoded['month'] +=1

In [92]:
data_encoded.to_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_all_data_encoded.csv', index=False)

In [93]:
%%time
cache = {}
data_encoded['item_shop_last_sale'] = -1
data_encoded['item_shop_last_sale'] = data_encoded['item_shop_last_sale'].astype(np.int8)
for idx, row in data_encoded.iterrows():    
    key = str(row.item_id)+' '+str(row.shop_id)
    if key not in cache:
        if row.item_cnt_month!=0:
            cache[key] = row.date_block_num
    else:
        last_date_block_num = cache[key]
        data_encoded.at[idx, 'item_shop_last_sale'] = row.date_block_num - last_date_block_num
        cache[key] = row.date_block_num 

Wall time: 5min 31s


In [94]:
cache = {}
data_encoded['item_last_sale'] = -1
data_encoded['item_last_sale'] = data_encoded['item_last_sale'].astype(np.int8)
for idx, row in data_encoded.iterrows():    
    key = row.item_id
    if key not in cache:
        if row.item_cnt_month!=0:
            cache[key] = row.date_block_num
    else:
        last_date_block_num = cache[key]
        if row.date_block_num>last_date_block_num:
            data_encoded.at[idx, 'item_last_sale'] = row.date_block_num - last_date_block_num
            cache[key] = row.date_block_num

In [95]:
data_encoded['item_shop_first_sale'] = data_encoded['date_block_num'] - data_encoded.groupby(['item_id','shop_id'])['date_block_num'].transform('min')
data_encoded['item_first_sale'] = data_encoded['date_block_num'] - data_encoded.groupby('item_id')['date_block_num'].transform('min')

In [96]:
def fill_na(df):
    for col in df.columns:
        if ('_lag_' in col) & (df[col].isnull().any()):
            if ('item_cnt' in col):
                df[col].fillna(0, inplace=True)         
    return df

data_encoded = fill_na(data_encoded)

In [97]:
data_encoded.to_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_all_data_encoded.csv', index=False)

# --------------------------------------------

In [1]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [2]:
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
nbrs = NearestNeighbors(n_neighbors=4, algorithm='ball_tree', metric='braycurtis').fit(X)
distances, indices = nbrs.kneighbors(X)

In [3]:
indices

array([[0, 1, 2, 3],
       [1, 0, 2, 3],
       [2, 1, 0, 3],
       [3, 4, 5, 2],
       [4, 3, 5, 2],
       [5, 4, 3, 2]], dtype=int64)

In [4]:
type(indices)

numpy.ndarray

In [5]:
np.save('data.npy', indices)

In [7]:
data = np.load('data.npy')

In [8]:
data

array([[0, 1, 2, 3],
       [1, 0, 2, 3],
       [2, 1, 0, 3],
       [3, 4, 5, 2],
       [4, 3, 5, 2],
       [5, 4, 3, 2]], dtype=int64)

In [117]:
distances

array([[0.        , 0.2       , 0.42857143, 1.        ],
       [0.        , 0.2       , 0.25      , 1.        ],
       [0.        , 0.25      , 0.42857143, 1.        ],
       [0.        , 0.2       , 0.42857143, 1.        ],
       [0.        , 0.2       , 0.25      , 1.        ],
       [0.        , 0.25      , 0.42857143, 1.        ]])

In [118]:
nbrs.kneighbors_graph(X).toarray()

array([[1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 0., 0.],
       [0., 0., 1., 1., 1., 1.],
       [0., 0., 1., 1., 1., 1.],
       [0., 0., 1., 1., 1., 1.]])